# Implementing self-healing autonomous software-defined IIoT-Edge networks for offshore wind power plants


Topic Summary:
> Software-defined Industrial Internet of Things -
Edge (IIoT-Edge) networks in offshore Wind Power Plants
(WPPs) enforce rigorous Service Level Agreements (SLAs) and
Quality of Service (QoS) requirements to ensure reliable trans-
mission of critical time-sensitive and best-effort data traffic. 

>On occasion, these IIoT-Edge networks experience flash events of
benign traffic flows which cause network congestion resulting
in intermittent network service interruptions. Further, given
the intricacies of the offshore environment, these IIoT-Edge
network equipment are exposed to extreme temperature profiles
which cause physical damage, degrade equipment performance,
and shorten their overall lifespan by 50%. 

>To mitigate these
two stochastic disruptions, this paper proposes an externally
adapted “Observe–Orient–Decide–Act” closed control loop self-
healing framework. This framework autonomously monitors
the operating temperature profiles of the IIoT-Edge network
equipment, detects traffic anomalies, and initiates corrective
actions through a threshold-triggered Deep Q-Network (DQN)
self-healing agent. These corrective actions reroute traffic to
avoid congested paths and overheated switches and throttle low-
priority traffic based on service type classification in extreme
cases.

Graphical Abstract:
![Graphical-Abstract](Decide_Module.png)

Importing relevant libraries:

In [1]:
import requests
import time
import logging
import pandas as pd
from time import strftime
from datetime import datetime
import os

## OBSERVE MODULE

#### 1. Connecting to the ONOS SDN Controller via RESTful APIs from the Knowledge Plane

In [2]:
'''
        ONOS SDN controller cluster interacting with the Knowledge Base
        Connecting to the primary SDN controller using a 8181 RESTful API based url and 
        with OAuth 2.0 access token for secure authentication and authorization.
'''


# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Prompting user for ONOS controller details
print("OBSERVE MODULE AS(n) OT Network Topology.\n\nDisclaimer: Press ENTER if you wish to use the default configurations.\n")
controller_ip = input("Enter the IP address of the ONOS controller (default is 192.168.0.6: )")
controller_ip = controller_ip if controller_ip else '192.168.0.6'
onos_base_url = f'http://{controller_ip}:8181/onos/v1'

username = input("Enter the username for ONOS controller (default is 'onos'): ")
username = username if username else 'onos'
password = input("Enter the password for ONOS controller (default is 'rocks'): ")
password = password if password else 'rocks'
onos_auth = (username, password)


OBSERVE MODULE AS(n) OT Network Topology.

Disclaimer: Press ENTER if you wish to use the default configurations.

Enter the IP address of the ONOS controller (default is 192.168.0.6: )
Enter the username for ONOS controller (default is 'onos'): 
Enter the password for ONOS controller (default is 'rocks'): 


#### 2. Reading the Control and Data Plane Metrics using RESTful API Requests

In [3]:
def get_devices():
    """ 
        Fetches network devices from the ONOS controller. 
    """
    
    try:
        response = requests.get(f'{onos_base_url}/devices', auth=onos_auth)
        if response.status_code == 200:
            devices = response.json().get('devices', [])
            df = pd.DataFrame(devices)
            return df
        else:
            print(f"Failed to retrieve Network devices: Status code {response.status_code} - {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching Network devices: {e}")

def get_links():
    """ 
        Fetches network links from the ONOS controller. 
    """
    
    try:
        response = requests.get(f'{onos_base_url}/links', auth=onos_auth)
        if response.status_code == 200:
            links = response.json().get('links', [])
            df = pd.DataFrame(links)
            return df
        else:
            print(f"Failed to retrieve Network links: Status code {response.status_code} - {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching Network Links: {e}")

def get_hosts():
    """ 
        Fetches network hosts from the ONOS controller. 
    """

    try:
        response = requests.get(f'{onos_base_url}/hosts', auth=onos_auth)
        if response.status_code == 200:
            hosts = response.json().get('hosts', [])
            df = pd.DataFrame(hosts)
            return df
        else:
            print(f"Failed to retrieve Network hosts: Status code {response.status_code} - {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching Network hosts: {e}")

def get_flows():
    """ 
        Fetches flow rules from the ONOS controller. 
    """
    
    try:
        response = requests.get(f'{onos_base_url}/flows', auth=onos_auth)
        if response.status_code == 200:
            flows = response.json().get('flows', [])
            df = pd.DataFrame(flows)
            return df
        else:
            print(f"Failed to retrieve Network flows: Status code {response.status_code} - {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching Network flows: {e}")


def get_port_statistics():
    """ 
        Fetches port statistics from all devices in the ONOS controller. 
    """
    
    try:
        response = requests.get(f'{onos_base_url}/statistics/ports', auth=onos_auth)
        if response.status_code == 200:
            port_stats = response.json().get('statistics', [])
            # Flatten the nested JSON structure for easier CSV writing
            flattened_stats = []
            for device in port_stats:
                device_id = device['device']
                for port in device['ports']:
                    port['device'] = device_id
                    flattened_stats.append(port)
            df = pd.DataFrame(flattened_stats)
            return df
        else:
            print(f"Failed to retrieve Port Statistics: Status code {response.status_code} - {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching network port statistics: {e}")


#For the paths
def get_devices_fp():
    response = requests.get(f'{onos_base_url}/devices', auth=onos_auth)
    return response.json()

# Function to get links
def get_links_fp():
    response = requests.get(f'{onos_base_url}/links', auth=onos_auth)
    return response.json()

#Determine the source to destination paths
def find_all_paths(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if node not in path:
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

# Generating the links between the source and destination paths
def src_to_dest_paths():
    devices = get_devices_fp()['devices']
    links = get_links_fp()['links']
    
    # Build adjacency list for the graph
    graph = {}
    for link in links:
        src = link['src']['device']
        dst = link['dst']['device']
        if src not in graph:
            graph[src] = []
        if dst not in graph:
            graph[dst] = []
        graph[src].append(dst)
        graph[dst].append(src)
    
    # Iterate over each pair of devices to find all paths
    device_ids = [device['id'] for device in devices]
    paths_list = []
    
    for i in range(len(device_ids)):
        for j in range(i + 1, len(device_ids)):
            source_device = device_ids[i]
            destination_device = device_ids[j]
            
            # Find all paths
            paths = find_all_paths(graph, source_device, destination_device)
            
            # Store paths
            for path in paths:
                paths_list.append({
                    'source': source_device,
                    'destination': destination_device,
                    'path': " -> ".join(path)
                })

    df_paths = pd.DataFrame(paths_list)
    return df_paths



In [4]:
devices = get_devices()
devices

,id,type,available,role,mfr,hw,sw,serial,driver,chassisId,lastUpdate,humanReadableLastUpdate,annotations
0,of:000000000000000c,SWITCH,True,MASTER,"Nicira, Inc.",Open vSwitch,2.17.9,None,ovs,c,1726559686282,connected 2h40m ago,"{'channelId': '192.168.0.5:59218', 'management..."
1,of:000000000000000d,SWITCH,True,MASTER,"Nicira, Inc.",Open vSwitch,2.17.9,None,ovs,d,1726559686123,connected 2h40m ago,"{'channelId': '192.168.0.5:59166', 'management..."
2,of:000000000000000a,SWITCH,True,MASTER,"Nicira, Inc.",Open vSwitch,2.17.9,None,ovs,a,1726559687133,connected 2h40m ago,"{'channelId': '192.168.0.5:59140', 'management..."
3,of:000000000000000b,SWITCH,True,MASTER,"Nicira, Inc.",Open vSwitch,2.17.9,None,ovs,b,1726559686547,connected 2h40m ago,"{'channelId': '192.168.0.5:59254', 'management..."
4,of:000000000000000e,SWITCH,True,MASTER,"Nicira, Inc.",Open vSwitch,2.17.9,None,ovs,e,1726559685813,connected 2h40m ago,"{'channelId': '192.168.0.5:59142', 'management..."
5,of:000000000000000f,SWITCH,True,MASTER,"Nicira, Inc.",Open vSwitch,2.17.9,None,ovs,f,1726559685964,connected 2h40m ago,"{'channelId': '192.168.0.5:59160', 'management..."
6,of:0000000000000010,SWITCH,True,MASTER,"Nicira, Inc.",Open vSwitch,2.17.9,None,ovs,10,1726559686771,connected 2h40m ago,"{'channelId': '192.168.0.5:59396', 'management..."
7,of:0000000000000011,SWITCH,True,MASTER,"Nicira, Inc.",Open vSwitch,2.17.9,None,ovs,11,1726559686613,connected 2h40m ago,"{'channelId': '192.168.0.5:59384', 'management..."
8,of:0000000000000014,SWITCH,True,MASTER,"Nicira, Inc.",Open vSwitch,2.17.9,None,ovs,14,1726559686660,connected 2h40m ago,"{'channelId': '192.168.0.5:59368', 'management..."
9,of:0000000000000015,SWITCH,True,MASTER,"Nicira, Inc.",Open vSwitch,2.17.9,None,ovs,15,1726559687012,connected 2h40m ago,"{'channelId': '192.168.0.5:59426', 'management..."


In [5]:
hosts = get_hosts()
hosts

,id,mac,vlan,innerVlan,outerTpid,configured,ipAddresses,locations
0,F6:F0:F9:ED:23:3D/None,F6:F0:F9:ED:23:3D,None,None,unknown,False,[10.0.1.54],"[{'elementId': 'of:0000000000000019', 'port': ..."
1,0E:7C:BE:87:07:F7/None,0E:7C:BE:87:07:F7,None,None,unknown,False,[10.0.1.56],"[{'elementId': 'of:000000000000001b', 'port': ..."
2,22:31:A9:0F:F8:6C/None,22:31:A9:0F:F8:6C,None,None,unknown,False,[10.0.1.77],"[{'elementId': 'of:000000000000001c', 'port': ..."
3,8A:EE:2C:FC:E3:DD/None,8A:EE:2C:FC:E3:DD,None,None,unknown,False,[10.0.1.53],"[{'elementId': 'of:0000000000000018', 'port': ..."
4,7E:E6:0A:29:85:98/None,7E:E6:0A:29:85:98,None,None,unknown,False,[10.0.1.43],"[{'elementId': 'of:0000000000000011', 'port': ..."
5,4E:37:62:EA:9F:74/None,4E:37:62:EA:9F:74,None,None,unknown,False,[10.0.1.33],"[{'elementId': 'of:0000000000000009', 'port': ..."
6,C6:33:FC:C1:9C:7E/None,C6:33:FC:C1:9C:7E,None,None,unknown,False,[10.0.1.14],"[{'elementId': 'of:000000000000000d', 'port': ..."
7,96:EB:16:57:80:48/None,96:EB:16:57:80:48,None,None,unknown,False,[10.0.1.66],"[{'elementId': 'of:0000000000000025', 'port': ..."
8,26:75:77:B4:06:BD/None,26:75:77:B4:06:BD,None,None,unknown,False,[10.0.1.51],"[{'elementId': 'of:0000000000000016', 'port': ..."
9,FE:58:A7:40:B2:88/None,FE:58:A7:40:B2:88,None,None,unknown,False,[10.0.1.67],"[{'elementId': 'of:0000000000000026', 'port': ..."


In [22]:
links =get_links()
links

,src,dst,type,state
0,"{'port': '1', 'device': 'of:0000000000000014'}","{'port': '4', 'device': 'of:0000000000000008'}",DIRECT,ACTIVE
1,"{'port': '2', 'device': 'of:0000000000000003'}","{'port': '1', 'device': 'of:0000000000000004'}",DIRECT,ACTIVE
2,"{'port': '3', 'device': 'of:0000000000000001'}","{'port': '1', 'device': 'of:0000000000000015'}",DIRECT,ACTIVE
3,"{'port': '1', 'device': 'of:0000000000000012'}","{'port': '2', 'device': 'of:0000000000000008'}",DIRECT,ACTIVE
4,"{'port': '3', 'device': 'of:0000000000000003'}","{'port': '1', 'device': 'of:0000000000000007'}",DIRECT,ACTIVE
...,...,...,...,...
73,"{'port': '1', 'device': 'of:0000000000000005'}","{'port': '2', 'device': 'of:0000000000000001'}",DIRECT,ACTIVE
74,"{'port': '1', 'device': 'of:0000000000000023'}","{'port': '8', 'device': 'of:0000000000000003'}",DIRECT,ACTIVE
75,"{'port': '3', 'device': 'of:0000000000000008'}","{'port': '1', 'device': 'of:0000000000000013'}",DIRECT,ACTIVE
76,"{'port': '1', 'device': 'of:0000000000000021'}","{'port': '6', 'device': 'of:0000000000000003'}",DIRECT,ACTIVE


In [24]:
flows = get_flows()
flows

,id,tableId,appId,groupId,priority,timeout,isPermanent,deviceId,state,life,packets,bytes,liveType,lastSeen,treatment,selector
0,281477504901495,0,org.onosproject.core,0,40000,0,True,of:000000000000000c,ADDED,9794,3001,126042,UNKNOWN,1726569482236,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'ETH_TYPE', 'ethType': ..."
1,281479100460205,0,org.onosproject.core,0,40000,0,True,of:000000000000000c,ADDED,9794,3159,439101,UNKNOWN,1726569482235,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'ETH_TYPE', 'ethType': ..."
2,281475624796078,0,org.onosproject.core,0,40000,0,True,of:000000000000000c,ADDED,9794,3159,439101,UNKNOWN,1726569482236,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'ETH_TYPE', 'ethType': ..."
3,281476292687096,0,org.onosproject.core,0,5,0,True,of:000000000000000c,ADDED,9794,844,82712,UNKNOWN,1726569482236,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'ETH_TYPE', 'ethType': ..."
4,281476618808057,0,org.onosproject.core,0,40000,0,True,of:000000000000000d,ADDED,9794,2963,124446,UNKNOWN,1726569482089,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'ETH_TYPE', 'ethType': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,281475411377083,0,org.onosproject.core,0,40000,0,True,of:0000000000000006,ADDED,9795,4665,195930,UNKNOWN,1726569482370,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'ETH_TYPE', 'ethType': ..."
160,281479019932926,0,org.onosproject.core,0,40000,0,True,of:0000000000000009,ADDED,9795,2929,123018,UNKNOWN,1726569482231,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'ETH_TYPE', 'ethType': ..."
161,281478887649794,0,org.onosproject.core,0,40000,0,True,of:0000000000000009,ADDED,9795,3161,439379,UNKNOWN,1726569482231,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'ETH_TYPE', 'ethType': ..."
162,281476295891648,0,org.onosproject.core,0,40000,0,True,of:0000000000000009,ADDED,9795,3161,439379,UNKNOWN,1726569482231,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'ETH_TYPE', 'ethType': ..."


In [25]:
#Separating the permanent (Switch to controller) flows from the temporary (switch-to-switch) flows
# For flows (permanent and temporary flows are identified)
dp_to_cp = flows[flows['appId'] == 'org.onosproject.core']  # Permanent
dp_to_dp = flows[flows['appId'] == 'org.onosproject.fwd']   # Renewed periodically/Temporary
dp_to_dp

,id,tableId,appId,groupId,priority,timeout,isPermanent,deviceId,state,life,packets,bytes,liveType,lastSeen,treatment,selector
37,6755402979620275,0,org.onosproject.fwd,0,10,10,False,of:0000000000000015,ADDED,13,1,98,UNKNOWN,1726569482969,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'IN_PORT', 'port': 1}, ..."
41,6755400945371156,0,org.onosproject.fwd,0,10,10,False,of:0000000000000015,ADDED,13,1,98,UNKNOWN,1726569482969,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'IN_PORT', 'port': 3}, ..."
124,6755402514497344,0,org.onosproject.fwd,0,10,10,False,of:0000000000000001,ADDED,13,1,98,UNKNOWN,1726569482929,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'IN_PORT', 'port': 3}, ..."
125,6755400114207690,0,org.onosproject.fwd,0,10,10,False,of:0000000000000001,ADDED,13,1,98,UNKNOWN,1726569482929,"{'instructions': [{'type': 'OUTPUT', 'port': '...","{'criteria': [{'type': 'IN_PORT', 'port': 4}, ..."


In [26]:
paths = src_to_dest_paths()
paths

,source,destination,path
0,of:000000000000000c,of:000000000000000d,of:000000000000000c -> of:0000000000000006 -> ...
1,of:000000000000000c,of:000000000000000d,of:000000000000000c -> of:0000000000000006 -> ...
2,of:000000000000000c,of:000000000000000d,of:000000000000000c -> of:0000000000000006 -> ...
3,of:000000000000000c,of:000000000000000d,of:000000000000000c -> of:0000000000000006 -> ...
4,of:000000000000000c,of:000000000000000a,of:000000000000000c -> of:0000000000000006 -> ...
...,...,...,...
14633,of:0000000000000006,of:0000000000000009,of:0000000000000006 -> of:0000000000000002 -> ...
14634,of:0000000000000006,of:0000000000000009,of:0000000000000006 -> of:0000000000000002 -> ...
14635,of:0000000000000006,of:0000000000000009,of:0000000000000006 -> of:0000000000000002 -> ...
14636,of:0000000000000006,of:0000000000000009,of:0000000000000006 -> of:0000000000000002 -> ...


In [27]:
port_stats = get_port_statistics()
port_stats

,port,packetsReceived,packetsSent,bytesReceived,bytesSent,packetsRxDropped,packetsTxDropped,packetsRxErrors,packetsTxErrors,durationSec,device
0,1,9165,7507,1028424,964788,0,0,0,0,9821,of:000000000000000c
1,2,558,8633,37224,999648,0,0,0,0,9821,of:000000000000000c
2,3,629,8704,40206,1002630,0,0,0,0,9821,of:000000000000000c
3,1,9149,7493,1027752,964396,0,0,0,0,9821,of:000000000000000d
4,2,572,8649,37812,1000516,0,0,0,0,9821,of:000000000000000d
...,...,...,...,...,...,...,...,...,...,...,...
133,3,7493,9149,964396,1027752,0,0,0,0,9821,of:0000000000000006
134,4,6931,8643,928220,993667,0,0,0,0,9821,of:0000000000000006
135,1,9132,7476,1027120,957344,0,0,0,0,9821,of:0000000000000009
136,2,566,8641,37560,993730,0,0,0,0,9821,of:0000000000000009


#### 3. Saving these static metrics to a csv file  (Portable for sharing)

In [19]:
# Create a timestamp
timestamp = strftime("%Y%m%d_%H%M%S")
file_name = f'Data/network_data_{timestamp}.xlsx'
with pd.ExcelWriter(file_name) as writer:
    devices.to_excel(writer, sheet_name='Devices')
    links.to_excel(writer, sheet_name='Links')
    hosts.to_excel(writer, sheet_name='Hosts')
    dp_to_dp.to_excel(writer, sheet_name='FD_to_FD_Flows')
    dp_to_cp.to_excel(writer, sheet_name='FD_to_SDNC_Flows')
    paths.to_excel(writer, sheet_name='Paths')

print(f"\nData successfully sent to Knowledge Base and saved to {file_name}")
       


Data successfully sent to Knowledge Base and saved to Data/network_data_20240917_123656.xlsx


In [ ]:
'''
        Setting up the InfluxDB Connection
        Connection via TCP/8086 with API Token Authentication
        Bucket name: onos_metrics
        Organization: Knowledge_Base
'''

# InfluxDB configuration
url = "http://192.168.0.7:8086"  #InfluxDB URL (Host Server)
token = "qkc2ZrwHen0ZzaPyBisN9E5bZYGNmhwO9R-ATu077_ieVy9ZqrhxqvHlzmn8zS2A5iCiBTGmSUM4hz9flPX6yg=="  # InfluxDB API Token
org = "Knowledge_Base"   #InfluxDB Organization
bucket = "Onos_Metrics"   #Influx Bucket Name


# Initialize InfluxDB client
client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org
)

write_api = client.write_api(write_options=SYNCHRONOUS)

In [ ]:
def current_network_state():
    devices = get_devices()
    links = get_links()
    hosts = get_hosts()
    flows = get_flows()
    port_stats = get_port_statistics()
    paths = src_to_dest_paths()
        
    return devices, links, hosts, flows, port_stats, paths

def write_dataframe_to_influx(df, measurement):
    """
        Writes a DataFrame to InfluxDB with a given measurement name.
    """
    
    points = []
    timestamp = strftime("%Y%m%d_%H%M%S")  # ISO 8601 format
    for _, row in df.iterrows():
        fields = {k: v for k, v in row.items() if pd.notnull(v) and not isinstance(v, (list, dict))}
        point = influxdb_client.Point(measurement).time(timestamp)
        for field_key, field_value in fields.items():
            point = point.field(field_key, field_value)
        points.append(point)
    write_api.write(bucket=bucket, org=org, record=points)

---

# Funding Information

<div style="display: flex; align-items: center;">
  <img src="https://upload.wikimedia.org/wikipedia/commons/b/b7/Flag_of_Europe.svg" alt="EU Logo" style="width: 150px; margin-right: 20px;">
  <img src="https://ec.europa.eu/research/mariecurieactions/sites/mariecurie2/files/marie_sklodowska-curie_actions.png" alt="Marie Curie Logo" style="width: 150px;">
</div>

The Innovative Tools for Cyber-Physical Energy Systems (InnoCyPES) project has received funding from the European Union’s Horizon 2020 research and innovation programme under the Marie Sklodowska Curie grant agreement No 956433.
